# Start Pooling

# Start Pooling

In [1]:
#import general packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import n_fct_t_rl_thm_ll as bond_fct
import prem_ibrd as prib



c:\Users\kaibe\miniforge3\envs\climada_env\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
bond_cache = {}

def get_bond_metrics(pool, pay_dam_pool_it, nominal_pool_it):
    pool_key = tuple(sorted(pool))  # Create a unique key for the pool
    if pool_key not in bond_cache:
        # If result isn't cached, compute and store it
        pay_dam_temp = {c: pay_dam_pool_it[c] for c in pool}
        nominal_temp = {c: nominal_pool_it[c] for c in pool}
        bond_metrics, returns, tot_coverage, premium_dic, nominal, es_metrics, MES_cty = bond_fct.mlt_cty_bond(
            countries=pool,
            pay_dam_df_dic=pay_dam_temp,
            nominals_dic=nominal_temp,
            opt_cap=True,
        )
        bond_cache[pool_key] = {
            "Returns": returns
            }
    return bond_cache[pool_key]

## Define Countries and Variables for Cat Bond Development

In [3]:
#choose country
countries = [212, 670]
countries_150 = []
countries_30 = [212,670]
#minimum damage to be protected per grid cell calculated using return period
lower_rp = 100

#number of simulated years
r = 10000

#set risk free rate, either single value or array
rf_rates = 0.00


#inititate IBRD premium calculation function
params_ibrd = prib.init_prem_ibrd(want_plot=False)
a, k, b = params_ibrd
#set risk muliplier reported by artems
artemis_multiplier = 4.11

#set sharpe ratio to beat
target_sharpe = 0.5

prot_share = 0.008
lower_share = 0.045
prot_rp = 250
lower_rp = 30

## Designe Single Country Bonds
### Define Cat Bond function

### Design Cat Bond for Country 1

In [4]:
bond_metrics_sng_dic = {}
returns_sng_dic = {}
premium_dic_sng_dic = {}
nominal_sng_dic = {}
pay_dam_df_sng_dic = {}
es_metrics_sng_dic = {}
int_grid_sng_dic = {}
imp_per_event_flt_sng_dic = {}
imp_admin_evt_flt_sng_dic = {}

for cty in countries:
    if cty in bond_metrics_sng_dic:
        print(f"Bond for {cty} already calculated, skipping.")
        continue
    print(f'Create bond for {cty}')
    if cty in countries_150:
        bond_metrics, returns, premium_dic, nominal, pay_dam_df, es_metrics, int_grid, imp_per_event_flt, imp_admin_evt_flt = bond_fct.sng_cty_bond(country=cty,
                                                                                                                                                    prot_rp=prot_rp, 
                                                                                                                                                    to_prot_share=lower_share,
                                                                                                                                                    buffer_distance_km=105,
                                                                                                                                                    res_exp=150,
                                                                                                                                                    grid_size=1000,
                                                                                                                                                    buffer_grid_size=3,
                                                                                                                                                    incl_plots=False)
    if cty in countries_30:
        bond_metrics, returns, premium_dic, nominal, pay_dam_df, es_metrics, int_grid, imp_per_event_flt, imp_admin_evt_flt = bond_fct.sng_cty_bond(country=cty,
                                                                                                                                                    prot_rp=prot_rp, 
                                                                                                                                                    to_prot_share=lower_share,
                                                                                                                                                    buffer_distance_km=105,
                                                                                                                                                    res_exp=30,
                                                                                                                                                    grid_size=1000,
                                                                                                                                                    buffer_grid_size=3,
                                                                                                                                                    incl_plots=False)
    bond_metrics_sng_dic[cty] = bond_metrics
    returns_sng_dic[cty] = returns
    premium_dic_sng_dic[cty] = premium_dic
    nominal_sng_dic[cty] = nominal
    pay_dam_df_sng_dic[cty] = pay_dam_df
    es_metrics_sng_dic[cty] = es_metrics
    int_grid_sng_dic[cty] = int_grid
    imp_per_event_flt_sng_dic[cty] = imp_per_event_flt
    imp_admin_evt_flt_sng_dic[cty] = imp_admin_evt_flt

Create bond for 212
Number of tracks in NA basin: 4904
The principal of the cat bond is: 190385817.244 [USD]
Principal as share of GDP: 0.378
Create bond for 670
Number of tracks in NA basin: 5476
The principal of the cat bond is: 307641548.326 [USD]
Principal as share of GDP: 0.354


## Designe Multi-Country Bond
### Set Variables for Multi-Country Bond

### Create Multi-Country bond

In [8]:
nominal_dic = {}
pay_dam_df_dic = {}
for cty in countries:
    nominal_dic[cty] = nominal_sng_dic[cty]
    pay_dam_df_dic[cty] = pay_dam_df_sng_dic[cty]
    
bond_metrics_pool, returns_pool, tot_coverage_pool, premium_dic_pool, nominal_pool, es_metrics_pool, MES_cty_pool = bond_fct.mlt_cty_bond(countries=countries,pay_dam_df_dic=pay_dam_df_dic,nominals_dic=nominal_dic,opt_cap=True)

In [6]:
el_pool = []
for cty in countries:
    el_pool.append(MES_cty_pool[cty]['EL'])

sum(el_pool)-premium_dic_pool['exp_loss']

0.0

In [ ]:
sng_ann_ret = {}
for cty in countries:
    sng_ann_ret[cty] = returns_sng_dic[cty]['Annual'][0] 

pool_ann_ret = returns_pool['Annual'][0]

In [ ]:
df_returns = pd.DataFrame({f"{country} Returns": returns for country, returns in sng_ann_ret.items()})

r = np.mean(df_returns,axis=0)

# Create a covariance matrix
covar = df_returns.cov()

In [ ]:
p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

num_assets = len(df_returns.columns)
num_portfolios = 10000

In [ ]:
from numpy.random import dirichlet

num_portfolios = 10000  # Number of portfolios to simulate
alpha = 1

for _ in range(num_portfolios):
    weights = dirichlet([alpha] * num_assets)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns = np.dot(weights, r) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(returns)
    var = covar.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # yearly standard deviation
    p_vol.append(sd)

In [ ]:
data = {'Returns':p_ret, 'Volatility':p_vol, 'Sharpe Ratio':np.array(p_ret)/np.array(p_vol)}

for counter, symbol in enumerate(df_returns.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights]

In [ ]:
portfolios  = pd.DataFrame(data)
max_sharpe_idx = portfolios['Sharpe Ratio'].idxmax()
max_sharpe_portfolio = portfolios.loc[max_sharpe_idx]
portfolios.head() # Dataframe of the 10000 portfolios created

In [ ]:
# Plot efficient frontier
portfolios.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio', cmap='viridis', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])
plt.text(max_sharpe_portfolio['Volatility'] + 0.001,max_sharpe_portfolio['Returns'],f"Max Sharpe: {max_sharpe_portfolio['Sharpe Ratio']:.2f}",fontsize=10,ha='left',va='center',color='red')

# Plot pool point
plt.scatter(np.std(pool_ann_ret), np.mean(pool_ann_ret), label='pool', color='purple', s=100)

# Add label for pool Sharpe ratio
plt.text(np.std(pool_ann_ret)+0.001,np.mean(pool_ann_ret),f'Sharpe: {np.mean(pool_ann_ret)/np.std(pool_ann_ret):.2f}',fontsize=10,ha='left',va='center',color='purple')

for cty in countries:
    plt.scatter(np.std(sng_ann_ret[cty]), np.mean(sng_ann_ret[cty]), label=cty, s=100)
plt.legend()

In [ ]:
closest_index = (portfolios['Volatility'] - np.std(pool_ann_ret)).abs().idxmin()

# Extract the corresponding return
closest_return = portfolios.loc[closest_index, 'Returns']
closest_volatility = portfolios.loc[closest_index, 'Volatility']
print(np.mean(pool_ann_ret) - closest_return)
print(np.std(pool_ann_ret) - closest_volatility)


In [ ]:
x = countries.copy()
x = [str(entry) for entry in x]
x.append('pool')
y = []
for cty in countries:
    y.append(bond_metrics_sng_dic[cty]['Total Premiums']/bond_metrics_sng_dic[cty]['Summed Payments'])
y.append(bond_metrics_pool['Total Premiums']/bond_metrics_pool['Summed Payments'])
plt.scatter(x,y)

In [ ]:
s = []
n = []
for cty in countries:
    s.append(bond_metrics_sng_dic[cty]['Total Premiums'])
    n.append(nominal_sng_dic[cty])

print(bond_metrics_pool['Total Premiums'][0]/np.sum(s))
print(nominal_pool/np.sum(n))